# Generative Adversarial Networks
GAN을 이용하여 새로운 패션 아이템을 만들어봅니다

In [1]:
import os
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets
from torchvision.utils import save_image

In [2]:
torch.manual_seed(1)    # reproducible

In [3]:
# Hyper Parameters
EPOCHS = 100
BATCH_SIZE = 100
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")
print("Using Device:", DEVICE)

Using Device: cpu


In [4]:
# Fashion MNIST digits dataset
trainset = datasets.FashionMNIST('./.data',
    train=True,
    download=True,
    transform=transforms.Compose([
       transforms.ToTensor(),
       transforms.Normalize((0.5,), (0.5,))
    ]))
train_loader = torch.utils.data.DataLoader(
    dataset     = trainset,
    batch_size  = BATCH_SIZE,
    shuffle     = True)

In [5]:
# Discriminator
D = nn.Sequential(
        nn.Linear(784, 256),
        nn.LeakyReLU(0.2),
        nn.Linear(256, 256),
        nn.LeakyReLU(0.2),
        nn.Linear(256, 1),
        nn.Sigmoid())

# Generator 
G = nn.Sequential(
        nn.Linear(64, 256),
        nn.ReLU(),
        nn.Linear(256, 256),
        nn.ReLU(),
        nn.Linear(256, 784),
        nn.Tanh())

In [6]:

# Device setting
D = D.to(DEVICE)
G = G.to(DEVICE)

# Binary cross entropy loss and optimizer
criterion = nn.BCELoss()
d_optimizer = optim.Adam(D.parameters(), lr=0.0002)
g_optimizer = optim.Adam(G.parameters(), lr=0.0002)

In [7]:
total_step = len(train_loader)
for epoch in range(EPOCHS):
    for i, (images, _) in enumerate(train_loader):
        images = images.reshape(BATCH_SIZE, -1).to(DEVICE)
        
        # Create the labels which are later used as input for the BCE loss
        real_labels = torch.ones(BATCH_SIZE, 1).to(DEVICE)
        fake_labels = torch.zeros(BATCH_SIZE, 1).to(DEVICE)

        # Train Discriminator

        # Compute BCE_Loss using real images where BCE_Loss(x, y): - y * log(D(x)) - (1-y) * log(1 - D(x))
        # Second term of the loss is always zero since real_labels == 1
        outputs = D(images)
        d_loss_real = criterion(outputs, real_labels)
        real_score = outputs
        
        # Compute BCELoss using fake images
        # First term of the loss is always zero since fake_labels == 0
        z = torch.randn(BATCH_SIZE, 64).to(DEVICE)
        fake_images = G(z)
        outputs = D(fake_images)
        d_loss_fake = criterion(outputs, fake_labels)
        fake_score = outputs
        
        # Backprop and optimize
        d_loss = d_loss_real + d_loss_fake
        d_optimizer.zero_grad()
        g_optimizer.zero_grad()
        d_loss.backward()
        d_optimizer.step()
        
        # Train Generator

        # Compute loss with fake images
        z = torch.randn(BATCH_SIZE, 64).to(DEVICE)
        fake_images = G(z)
        outputs = D(fake_images)
        
        # We train G to maximize log(D(G(z)) instead of minimizing log(1-D(G(z)))
        # For the reason, see the last paragraph of section 3. https://arxiv.org/pdf/1406.2661.pdf
        g_loss = criterion(outputs, real_labels)
        
        # Backprop and optimize
        d_optimizer.zero_grad()
        g_optimizer.zero_grad()
        g_loss.backward()
        g_optimizer.step()
        
        if (i+1) % 200 == 0:
            print('Epoch [{}/{}], Step [{}/{}], d_loss: {:.4f}, g_loss: {:.4f}, D(x): {:.2f}, D(G(z)): {:.2f}' 
                  .format(epoch, EPOCHS, i+1, total_step, d_loss.item(), g_loss.item(), 
                          real_score.mean().item(), fake_score.mean().item()))

Epoch [0/100], Step [200/600], d_loss: 0.0660, g_loss: 4.2427, D(x): 0.99, D(G(z)): 0.05
Epoch [0/100], Step [400/600], d_loss: 0.1080, g_loss: 5.2820, D(x): 0.98, D(G(z)): 0.08
Epoch [0/100], Step [600/600], d_loss: 0.0489, g_loss: 4.7447, D(x): 0.98, D(G(z)): 0.03
Epoch [1/100], Step [200/600], d_loss: 0.0854, g_loss: 3.9816, D(x): 0.98, D(G(z)): 0.05
Epoch [1/100], Step [400/600], d_loss: 0.0284, g_loss: 5.3748, D(x): 0.99, D(G(z)): 0.02
Epoch [1/100], Step [600/600], d_loss: 0.1014, g_loss: 5.9137, D(x): 0.96, D(G(z)): 0.01
Epoch [2/100], Step [200/600], d_loss: 0.0550, g_loss: 5.2941, D(x): 0.98, D(G(z)): 0.02
Epoch [2/100], Step [400/600], d_loss: 0.0316, g_loss: 5.7140, D(x): 0.99, D(G(z)): 0.01
Epoch [2/100], Step [600/600], d_loss: 0.0235, g_loss: 6.4996, D(x): 0.99, D(G(z)): 0.01
Epoch [3/100], Step [200/600], d_loss: 0.0037, g_loss: 12.1831, D(x): 1.00, D(G(z)): 0.00
Epoch [3/100], Step [400/600], d_loss: 0.1413, g_loss: 8.5847, D(x): 0.94, D(G(z)): 0.01
Epoch [3/100], Step 

KeyboardInterrupt: 

## 참고
본 튜토리얼은 다음 자료를 참고하여 만들어졌습니다.

* [yunjey/pytorch-tutorial](https://github.com/yunjey/pytorch-tutorial) - MIT License